In [ ]:
# Import matplotlib before seaborn
from datetime import datetime
import matplotlib as mpl
import matplotlib.pyplot as plt
import itertools  # for color palette cycling
import os
import re
import pandas as pd
import seaborn as sns
import sys
from cycler import cycler
import seaborn as sns
%matplotlib inline

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import networkx as nx
import numpy as np

In [ ]:
sys.path.append('/work/rnaseq/pcor_new/networkx')
import networkx_helpers as nxh
import networkx_explore as nxe

In [ ]:
plot_dir = '170417_results_across_networks'
if not os.path.exists(plot_dir):
    os.mkdir(plot_dir)

In [ ]:
! ls

In [ ]:
import glob

In [ ]:
! ls cutoff_0.1/results/data/*_edges.tsv

In [ ]:
glob.glob('cutoff_*/results/data/*_edges.tsv')

In [ ]:
def get_network_edge_paths():
    files = glob.glob('cutoff_*/results/data/*_edges.tsv')
    networks = pd.DataFrame({'path':files})
    networks['cutoff'] = networks['path'].str.extract('cutoff_([.0-9]+)', expand=True)
    networks['cutoff'] = networks['cutoff'].astype(float)
    return networks

nw_info = get_network_edge_paths()
nw_info

In [ ]:
#n_toy = nxh.build_network('cutoff_0.1/results/data/input_to_R--no_features_dropped--top_41328_edges.tsv')
n_toy = nxh.build_network('cutoff_0.01/results/data/input_to_R--no_features_dropped--top_1e+06_edges.tsv')

In [ ]:
from functools import partial

In [ ]:
partial(nx.spring_layout, weight='pcor')

In [ ]:
methylotrophy_genes = ['ethane', 'Methanol', 'hexulose-6-phosphate']
methylotrophy_nodes = nxe.get_nodes_including_list_of_strings(n_toy, methylotrophy_genes)
#mSG = nx.Graph(methylotrophy_nodes)
#methylotrophy_network = nx.Graph(n_toy(mSG.nodes()))
methylotrophy_network = n_toy.subgraph(methylotrophy_nodes)
mnt = nxe.subgraph_by_cutoff(methylotrophy_network, cutoff = 0.0)
plt.figure(figsize=(8,8))
nxe.draw(mnt, layout=partial(nx.spring_layout, scale = 500), edge_multiplier=500) #, weight='pcor'))

In [ ]:
node_strings = ['Phage', 'phage']
nodes = nxe.get_nodes_including_list_of_strings(n_toy, node_strings)
string_network = n_toy.subgraph(nodes)
snt = nxe.subgraph_by_cutoff(string_network, cutoff = 0.0)
nxe.draw(snt, layout=partial(nx.spring_layout, scale = 600), edge_multiplier=100) #, weight='pcor')) 

In [ ]:
node_strings = ['ethane', 'Methanol', 'hexulose-6-phosphate', 'Phage', 'phage']
nodes = nxe.get_nodes_including_list_of_strings(n_toy, node_strings)
string_network = n_toy.subgraph(nodes)
snt = nxe.subgraph_by_cutoff(string_network, cutoff = 0.1)
nxe.draw(snt, layout=partial(nx.spring_layout, scale = 100), edge_multiplier=50) #, weight='pcor')) 

In [ ]:
high_pcors_w_hypo = nxe.summarise_sum_abs_pcors(n_toy)
high_pcors = high_pcors_w_hypo[ ~ high_pcors_w_hypo['product'].str.contains('hypothetical')]
high_pcors.head(15)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(4, 2.5))
x = 'sum(abs(pcor))'
high_pcors[x].hist(ax=ax)
ax.set_xlabel(x)
ax.set_ylabel('# of nodes')